In [322]:
# import os
# from datetime import datetime
# from dotenv import load_dotenv
# #
# # #--------Google Drive Integration--------#
# # # from google.colab import drive, userdata
# # # This gives Colab access to your files in Google Drive.
# # # drive.mount('/content/drive')
# # # 'GITHUB_USERNAME' and 'GITHUB_TOKEN' saved as secrets in Colab.
# # GITHUB_USERNAME = userdata.get('GITHUB_USERNAME')
# # GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
# # REPOSITORY_NAME = 'PyNucleus-Model' # Your repository name
# # NOTEBOOK_DRIVE_PATH = "/content/drive/MyDrive/PyNucleus Project/Capstone Project.ipynb"
# #
# #
# # #--------Cursor Integration--------#
# # # Load environment variables from .env file
# load_dotenv()
# #
# # # Get GitHub credentials from environment variables
# GITHUB_USERNAME = os.getenv('GITHUB_USERNAME')
# GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
# #
# # # Print to verify the variables are loaded (remove this in production)
# print(f"Username: {GITHUB_USERNAME}")
# print(f"Token: {GITHUB_TOKEN[:4]}...") # Only print first 4 chars of token for security
# #
# # Repository information
# REPOSITORY_NAME = 'PyNucleus-Model'
# NOTEBOOK_REPO_FILENAME = "Capstone Project.ipynb"
# LOG_FILENAME = "update_log.txt"

# # Pull latest changes from GitHub
# print("Pulling latest changes from GitHub...")
# !git pull https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git main

# print("Repository is up to date!")

# # Log start time
# with open("update_log.txt", "a") as f:
#     f.write(f" {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Log Update\n")

In [323]:
# PyNucleus Model - Setup and Imports
import sys
from pathlib import Path

# Add src to Python path
src_path = str(Path().resolve() / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import PyNucleus Pipeline modules
from pynucleus.pipeline import RAGPipeline, DWSIMPipeline, ResultsExporter, PipelineUtils
from pynucleus.integration.config_manager import ConfigManager
from pynucleus.integration.dwsim_rag_integrator import DWSIMRAGIntegrator
from pynucleus.integration.llm_output_generator import LLMOutputGenerator


print(" PyNucleus Model - Pipeline Ready!")
print(" Available Components:")
print("   • RAGPipeline - Document processing and retrieval")
print("   • DWSIMPipeline - Chemical process simulation") 
print("   • ResultsExporter - CSV export functionality")
print("   • PipelineUtils - Complete pipeline orchestration")

 PyNucleus Model - Pipeline Ready!
 Available Components:
   • RAGPipeline - Document processing and retrieval
   • DWSIMPipeline - Chemical process simulation
   • ResultsExporter - CSV export functionality
   • PipelineUtils - Complete pipeline orchestration


# **PyNucleus Model - Complete Pipeline**

This notebook contains the complete PyNucleus model pipeline with separate sections for:
1. **Data Ingestion and Preprocessing for RAG** 
2. **DWSIM Integration and Simulation**
3. **Results Export to CSV**

In [324]:
# Initialize Pipeline Components
pipeline = PipelineUtils(results_dir="data/05_output/results")


print("\n🔧 Pipeline Initialized!")
print("Available Functions:")
print("   • pipeline.run_complete_pipeline() - Run everything")
print("   • pipeline.run_rag_only() - RAG pipeline only")  
print("   • pipeline.run_dwsim_only() - DWSIM simulations only")
print("   • pipeline.quick_test() - Verify status")
print("   • pipeline.view_results_summary() - View results")
print("   • pipeline.print_pipeline_status() - Detailed status")
print("   • pipeline.clean_all_results() - Clean all data")


🔧 Setting up RAG imports...
✅ RAG imports ready!
🔧 Setting up DWSIM imports...
✅ DWSIM modules imported successfully
📁 Results directory: data/05_output/results
🔧 Pipeline Utils initialized with results dir: data/05_output/results

🔧 Pipeline Initialized!
Available Functions:
   • pipeline.run_complete_pipeline() - Run everything
   • pipeline.run_rag_only() - RAG pipeline only
   • pipeline.run_dwsim_only() - DWSIM simulations only
   • pipeline.quick_test() - Verify status
   • pipeline.view_results_summary() - View results
   • pipeline.print_pipeline_status() - Detailed status
   • pipeline.clean_all_results() - Clean all data


In [325]:
# ========================================
# SECTION 1: COMPLETE PIPELINE - Run Everything (Basic Mode)
"""This section is the basic pipeline which covers 
- Runs standard RAG + DWSIM + CSV Export
- Output: Basic CSV files with simulation results
- For: Regular users who need standard functionality """
# ========================================

# Run the complete pipeline (RAG + DWSIM + Export)
results = pipeline.run_complete_pipeline()

# Display results summary
if results:
    print(f"\n🎉 Pipeline completed in {results['duration']:.1f} seconds!")
    print(f"📊 RAG Results: {len(results['rag_data'])} queries processed")
    print(f"🔬 DWSIM Results: {len(results['dwsim_data'])} simulations completed")
    print(f"📁 Exported Files: {len(results['exported_files'])} CSV files created")
else:
    print("❌ Pipeline execution failed")

🚀 Running complete PyNucleus pipeline...
🗑️ RAG results cleared.
🗑️ DWSIM results cleared.
📚 Starting RAG Pipeline...
Step 1: Processing source documents...
--- 📄 Starting processing for 5 file(s) in '/Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/source_documents' ---


Processing files:   0%|          | 0/5 [00:00<?, ?it/s]WARNING:unstructured:libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


 ▶ Processing: Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.docx
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/02_processed/converted_to_txt/Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.txt
 ▶ Processing: mcp_basics.txt
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/02_processed/converted_to_txt/mcp_basics.txt
 ▶ Processing: feasibility_factors.txt
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/02_processed/converted_to_txt/feasibility_factors.txt
 ▶ Processing: Bist_Madan.pdf


Processing files: 100%|██████████| 5/5 [00:00<00:00, 11.51it/s]

   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/02_processed/converted_to_txt/Bist_Madan.txt
 ▶ Processing: sample_document.txt
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/02_processed/converted_to_txt/sample_document.txt

 All files processed.

Step 2: Scraping Wikipedia articles...
🔍 Starting Wikipedia article search for 5 keywords...
▶️  Searching for: modular design


✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_modular_design.txt
▶️  Searching for: software architecture
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_software_architecture.txt
▶️  Searching for: system design
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_system_design.txt
▶️  Searching for: industrial design
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_industrial_design.txt
▶️  Searching for: supply chain


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_supply_chain.txt

✨ Article scraping complete!

Step 3: Processing and chunking documents...
📰 Found 5 Wikipedia articles
📄 Found 5 converted documents
📋 Total documents loaded: 10
✂️ Split into 846 chunks

✅ Successfully saved chunked data to /Users/mohammadalmusaiteer/PyNucleus-Model/data/03_intermediate/converted_chunked_data/:
  • chunked_data_full.json - Complete data with metadata
  • chunked_data_stats.json - Statistical analysis
  • chunked_data_content.txt - Human-readable content


Step 4: Building FAISS vector store...
=== FAISS VectorDB Analysis ===
Started: 2025-06-11 01:26:01
Loaded 846 documents from /Users/mohammadalmusaiteer/PyNucleus-Model/data/03_intermediate/converted_chunked_data/chunked_data_full.json
Embedding device → cpu   | dim=384
Docs indexed : 846
Index file   : /Users/mohammadalmusaiteer/PyNucleus-Model/data/04_models/chunk_reports/pynucleus_mcp.faiss
Embeds .

In [326]:
# # ========================================
# # OPTIONAL: INDIVIDUAL PIPELINE COMPONENTS
# # ========================================

# # Option 1: Run only RAG Pipeline
# print("📚 RAG Only Pipeline:")
# rag_results = pipeline.run_rag_only()
# if rag_results:
#     print(f"   ✅ {len(rag_results['rag_data'])} RAG queries processed")

# print("\n" + "="*50 + "\n")

# # Option 2: Run only DWSIM Simulations  
# print("🔬 DWSIM Only Pipeline:")
# dwsim_results = pipeline.run_dwsim_only()
# if dwsim_results:
#     print(f"   ✅ {len(dwsim_results['dwsim_data'])} DWSIM simulations completed")

In [327]:
# # ========================================
# # OPTIONAL: View Results & Status (After Running Pipelines)
# # ========================================

# # View pipeline status
# pipeline.print_pipeline_status()

# print("\n" + "="*50 + "\n")

# # View results summary
# pipeline.view_results_summary()

# print("\n" + "="*50 + "\n")

# # Quick test
# test_results = pipeline.quick_test()
# print(f"✅ Quick test completed! Found {test_results['csv_files_count']} CSV files")

In [328]:
# ========================================
# OPTIONAL: CLEANUP AND RESET (Optional)
# ========================================
# Uncomment the lines below if you need to clean up results:

# Clean up and reset (optional - removes all previous results)
# pipeline.clean_all_results()

# print("✅ PyNucleus Pipeline is ready!")
# print("📋 Usage Guide:")
# print("   • Section 1: Run complete pipeline (RAG + DWSIM + Export)")
# print("   • Section 2: Individual components (commented out)")
# print("   • Section 3: Utility functions (status, summary, test)")
# print("   • Enhanced Pipeline: Advanced configuration, integration, LLM output")
# print("   • Enhanced Features: Configuration, Integration, LLM Output")
# print("\n🔄 Run any cell multiple times to re-execute components")
# print("📁 All results automatically saved as CSV files in data/05_output/results/")

In [329]:
# ========================================
# OPTION A: Run Individual Components (Alternative to Section 1)
# ========================================
# Uncomment ONLY the lines you want to run:

# Option A1: Complete pipeline (same as Section 1)
# results = pipeline.run_complete_pipeline()

# Option A2: Individual components only
# rag_results = pipeline.run_rag_only()        # RAG documents only
# dwsim_results = pipeline.run_dwsim_only()    # DWSIM simulations only

In [330]:
# ========================================
# ENHANCED PIPELINE - Initialize Advanced Features
"""PREREQUISITE: Run Section 1 (Complete Pipeline) first!
This section adds advanced capabilities ON TOP OF the basic pipeline:
• Financial analysis with ROI calculations
• LLM-ready reports and summaries  
• DWSIM-RAG integration with enhanced analytics
• Custom configuration templates"""
# ========================================

print("🔧 Initializing Enhanced Pipeline Components...")

try:
    # Force reload modules to get latest version (fixes notebook caching)
    import importlib
    
    # Clear any cached modules
    modules_to_reload = [
        'pynucleus.integration.config_manager',
        'pynucleus.integration.dwsim_rag_integrator', 
        'pynucleus.integration.llm_output_generator'
    ]
    
    for module_name in modules_to_reload:
        if module_name in sys.modules:
            importlib.reload(sys.modules[module_name])
    
    config_manager = ConfigManager(config_dir="configs")
    dwsim_rag_integrator = DWSIMRAGIntegrator(
    rag_pipeline=pipeline.rag_pipeline if hasattr(pipeline, 'rag_pipeline') else None,
    results_dir="data/05_output/results"
    )
    llm_generator = LLMOutputGenerator(results_dir="data/05_output/llm_reports")

    print("✅ Enhanced Pipeline Ready: Configuration, Integration, LLM Output")
    print(f"✅ LLM reports will be saved to: {llm_generator.results_dir}/")
    enhanced_available = True
    
except Exception as e:
    print(f"⚠️ Enhanced features not available: {e}")
    import traceback
    traceback.print_exc()
    enhanced_available = False

🔧 Initializing Enhanced Pipeline Components...
✅ Enhanced Pipeline Ready: Configuration, Integration, LLM Output
✅ LLM reports will be saved to: data/05_output/llm_reports/


In [331]:
# ENHANCED PIPELINE STEP 2: Configuration Templates
if 'enhanced_available' in locals() and enhanced_available:
    json_template = config_manager.create_template_json("simulation_config_template.json", verbose=True)
    csv_template = config_manager.create_template_csv("simulation_config_template.csv", verbose=True)
    
    print("✅ Configuration templates created:")
    print(f"   JSON: {json_template}")
    print(f"   CSV: {csv_template}")
else:
    print("❌ Enhanced configuration not available")
    print("⚠️ Run Cell 10 (Enhanced Pipeline Initialization) first!")

⏭️ Template already exists, skipping: configs/simulation_config_template.json
⏭️ Template already exists, skipping: configs/simulation_config_template.csv
✅ Configuration templates created:
   JSON: configs/simulation_config_template.json
   CSV: configs/simulation_config_template.csv


In [332]:
# ENHANCED PIPELINE STEP 3: DWSIM-RAG Integration with Enhanced Analytics
if 'enhanced_available' in locals() and enhanced_available:
    dwsim_results = pipeline.dwsim_pipeline.get_results()
    
    if dwsim_results:
        # Perform integration
        integrated_results = dwsim_rag_integrator.integrate_simulation_results(
            dwsim_results, perform_rag_analysis=True
        )
        
        # Export results
        integrated_export_file = dwsim_rag_integrator.export_integrated_results()
        
        # Show key metrics only
        if integrated_results:
            sample = integrated_results[0]
            print(f"✅ Enhanced Analysis Complete:")
            print(f"   Simulations: {len(integrated_results)}")
            print(f"   Performance: {sample['performance_metrics']['overall_performance']}")
            print(f"   Efficiency: {sample['performance_metrics']['efficiency_rating']}")
            print(f"   Results: {integrated_export_file}")
    else:
        print("⚠️ No DWSIM results available")
else:
    print("❌ Enhanced integration not available")
    print("⚠️ Run Cell 10 (Enhanced Pipeline Initialization) first!")

✅ Enhanced 5 simulations with RAG insights
✅ Enhanced Analysis Complete:
   Simulations: 5
   Performance: Good
   Efficiency: High
   Results: data/05_output/results/integrated_dwsim_rag_results_20250611_012607.json


In [333]:
# ENHANCED PIPELINE STEP 4: LLM-Ready Output with Enhanced Feed Conditions
if 'enhanced_available' in locals() and enhanced_available and 'integrated_results' in locals():
    print("🔄 Running enhanced LLM output generation with detailed feed conditions...")
    # Generate LLM summary for each simulation using Jinja2 template
    llm_ready_files = []
    for i, result in enumerate(integrated_results):
        # Template uses original_simulation.case_name, no need to add simulation_name
        llm_file = llm_generator.export_llm_ready_text(result)
        llm_ready_files.append(llm_file)
    
    # Export financial analysis and show metrics
    financial_file = llm_generator.export_financial_analysis(integrated_results)
    metrics = llm_generator._calculate_key_metrics(integrated_results)
    
    print(f"✅ Analysis Reports Generated:")
    print(f"   LLM Summaries: {len(llm_ready_files)} files created")
    for llm_file in llm_ready_files:
        print(f"     • {llm_file}")
    print(f"   Financial Analysis: {financial_file}")
    print(f"\n💰 Key Financial Metrics:")
    print(f"   Recovery Rate: {metrics['avg_recovery']:.1f}%")
    print(f"   Daily Revenue: ${metrics['estimated_revenue']:,.2f}")
    print(f"   Daily Profit: ${metrics['net_profit']:,.2f}")
    print(f"   ROI: {metrics['roi']:.1f}%")
    
elif 'enhanced_available' in locals() and enhanced_available:
    print("❌ Run Cell 12 (DWSIM-RAG Integration) first to generate integrated_results")
else:
    print("❌ Enhanced LLM output not available")
    print("⚠️ Run Cell 10 (Enhanced Pipeline Initialization) first!")

🔄 Running enhanced LLM output generation with detailed feed conditions...
✅ Analysis Reports Generated:
   LLM Summaries: 5 files created
     • data/05_output/llm_reports/distillation_ethanol_water_summary.md
     • data/05_output/llm_reports/reactor_methane_combustion_summary.md
     • data/05_output/llm_reports/heat_exchanger_steam_summary.md
     • data/05_output/llm_reports/absorber_co2_capture_summary.md
     • data/05_output/llm_reports/crystallizer_salt_summary.md
   Financial Analysis: data/05_output/llm_reports/financial_analysis_20250611_012607.csv

💰 Key Financial Metrics:
   Recovery Rate: 82.5%
   Daily Revenue: $148,500.00
   Daily Profit: $58,500.00
   ROI: 6.5%


In [334]:
# ========================================
# JINJA2 PROMPT TEMPLATE SYSTEM
# ========================================

# Simple one-line integration
exec(open('prompts/notebook_integration.py').read())

# Quick demo and validation
demo_prompts()
validate_prompts()

# Example usage
prompt = create_prompt(
    question="What troubleshooting steps would you recommend for a distillation column showing efficiency issues?",
    system_msg="You are an expert chemical process engineer.",
    context="A distillation column is showing efficiency issues.",
    constraints="Safety first, cost-effective solutions",
    format_instructions="Provide a structured response with causes and solutions."
)

print("\n📋 Example Prompt:")
print("─" * 30)
print(prompt[:200] + "..." if len(prompt) > 200 else prompt)

print("\n✅ Prompt system ready!")

✅ Prompt system initialized - Template loaded from: /Users/mohammadalmusaiteer/PyNucleus-Model/prompts
✅ Prompt system loaded successfully!
📚 Available functions:
   • create_prompt(question, system_msg=None, context=None, **kwargs)
   • demo_prompts() - Show example prompts
   • validate_prompts() - Test system functionality
   • integrate_pynucleus_results(results) - Generate from simulation data
   • save_prompt(prompt, filename) - Save to file
🧪 PyNucleus Prompt System Demo

📋 Example 1: Process Troubleshooting
────────────────────────────────────────



<SYSTEM>
You are an expert chemical process engineer specializing in distillation optimization.

CONSTRAINTS:
Consider safety protocols, environmental regulations, and cost-effectiveness in all recommendations.

OUTPUT FORMAT:
Provide response as: 1) Potential causes (ranked by probability), 2) Diagnostic steps, 3) Corrective actions with expected outcomes.
</SYSTEM>


<CONTEXT>
A methanol-water distillation column is showing decli

In [ ]:
# """Before you do version control, remove all files in the data directory"""

# # In Capstone Project.ipynb
# pipeline.clean_all_results()  # This will clean all previous results
# results = pipeline.run_complete_pipeline()  # Run fresh test

In [336]:
# ========================================
# VERSION CONTROL (Optional - For Maintainers Only)
# ========================================
# Uncomment the lines below if you need to update the repository:

from datetime import datetime

# Log end time
with open("update_log.txt", "a") as f:
    f.write(f"\n {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} changes made and pushed to origin main\n")

# Simple GitHub update function
def update_github():
    print(" Starting GitHub update...")
    !git add .
    print("📦 Files added to staging")
    !git commit -m "Update: $(date +'%Y-%m-%d %H:%M:%S')"
    print("💾 Changes committed")
    !git push origin main
    print("✅ Changes pushed to GitHub successfully!")

# To use it, just run:
update_github()

 Starting GitHub update...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


📦 Files added to staging
[main 9d0c20f] Update: 2025-06-11 01:26:38
 15 files changed, 1112 insertions(+), 309 deletions(-)
 delete mode 100644 configs/my.json
 delete mode 100644 configs/system_diagnostic_test.csv
 delete mode 100644 configs/system_diagnostic_test.json
 rename logs/{diagnostic_report_20250611_004838.txt => diagnostic_report_20250611_011514.txt} (84%)
 delete mode 100644 logs/pipeline.log
 delete mode 100644 logs/system_diagnostic_20250611_004838.log
 create mode 100644 logs/system_diagnostic_20250611_011514.log
 create mode 100644 prompts/README.md
 create mode 100644 prompts/notebook_integration.py
 create mode 100644 prompts/prompt_system.py
 create mode 100644 prompts/qwen_prompt.j2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


💾 Changes committed


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Enumerating objects: 24, done.
Counting objects: 100% (24/24), done.
Delta compression using up to 8 threads
Compressing objects: 100% (16/16), done.
Writing objects: 100% (16/16), 15.11 KiB | 15.11 MiB/s, done.
Total 16 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (7/7), completed with 7 local objects.
To https://github.com/Saytor20/PyNucleus-Model.git
   9aabc26..9d0c20f  main -> main
✅ Changes pushed to GitHub successfully!
